In [1]:
import urllib2
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import os
import sys

In [22]:
#51004 done
readin = ['51002','51003','51101','51001','51000','51WHO','51208',
         'NWWH1','51201','51204','OOUHI','MOKH1','51210','51202','51203','51205'
         ,'KLIH1','KWHH1','51407','ILOH1','51206']

In [3]:
hist_data_url = "http://www.ndbc.noaa.gov/historical_data.shtml"
main_url = "http://www.ndbc.noaa.gov"
data_folder = "python_test/metocean_data"

In [23]:
def readin_ndbc_stdmet(no_buoy):
    req_1 = urllib2.urlopen(hist_data_url)
    soup_1 = BeautifulSoup(req_1)
    a_tags = soup_1.find_all("a",attrs = {'href':re.compile(".*"+no_buoy+
                                                          ".*stdmet.*")})
    print "number of years of the data for %s buoy is %i" %(no_buoy, len(a_tags))
    for i in range(len(a_tags)):
        link = a_tags[i].get('href')
        req_2 = urllib2.urlopen(main_url+link)
        soup_2 = BeautifulSoup(req_2)
        data_tag = soup_2.find_all("a",attrs = {'href':re.compile(".*"+no_buoy+\
                                                ".*txt.*stdmet.*")})
        data_link = data_tag[0].get('href')
        req_3 = urllib2.urlopen(main_url+data_link)
        data_str = req_3.read()
        if sys.version_info[0]<3:
            from StringIO import StringIO
        else:
            from io import StringIO
        data_csv = StringIO(data_str)
        data_df = pd.read_csv(data_csv,delim_whitespace = True)
        year = data_df.iat[2,0]
        file_path = '/'+data_folder+'/'+no_buoy+'/'+str(year)+'.csv'
        directory = os.path.dirname(file_path)
        
        try:
            os.stat(directory)
        except:
            os.mkdir(directory)
            
        data_df.to_csv(file_path)
        time.sleep(1)
    print "%s buoy is done" %no_buoy
        

In [5]:
for i in range(len(readin)):
    readin_ndbc_stdmet(readin[i])
    if i in (3,6,9,12,15,18,21):
        time.sleep(10)
    time.sleep(3)

d:\Anaconda2\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


number of years of the data for 51004 buoy is 32


KeyError: 'YY'